In [19]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = {}
y = {}
X["train"] = np.load("./train_feature_save.npy",allow_pickle=True )[:9025]
y["train"] = np.load("./train_label_save.npy").squeeze()[:9025]
X["dev"] = np.load("./test_feature_save.npy")[:1003]
y["dev"] = np.load("./test_label_save.npy").squeeze()[:1003]
X_text_train = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_emotion/X_train_text.npy")
X_text_dev = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_emotion/X_test_text.npy")
X["train"].shape, X["dev"].shape, y["dev"].shape

((9025, 768), (1003, 768), (1003,))

In [20]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=0.1)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.8005982053838484

In [21]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((9025, 769), (1003, 769))

In [22]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 10 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [23]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1003, 9025)

In [24]:
ls

CSV_essay_bertlin_alg1.ipynb  model_save_feature/
Smallest_k.ipynb              new_predictions.npy
Smallest_k.py                 old_predictions.npy
Use_approK.ipynb              run_retrain.sh
__pycache__/                  run_tagger.sh
appro_ks.npy                  save_feature.py
appro_ks_IP.npy               save_feature.sh
bert_util.py                  test_feature_save.npy
essay_data/                   test_label_save.npy
essay_dev.tsv                 train_feature_save.npy
essay_train.tsv               train_label_save.npy


In [25]:
app_k = np.array(np.load("appro_ks_IP.npy", allow_pickle=True).squeeze())
pred = np.load("old_predictions.npy", allow_pickle=True)
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if app_k[i] == None:
        app_k_nonon.append(X["train"].shape[0])
    else:
        app_k_nonon.append(app_k[i])

19
2708
458
None
897
32
None
24
786
101
27
486
None
236
312
290
1099
653
390
41
644
414
779
118
None
714
2043
1244
225
26
640
202
49
None
None
13
2195
102
37
387
445
126
2155
715
1982
43
1108
91
311
9
579
None
1008
692
74
375
266
509
92
98
112
92
661
782
16
None
62
411
None
37
None
1267
13
None
151
489
6
44
3605
54
82
828
401
3188
111
12
752
3
3968
None
None
1627
899
430
1931
None
None
None
1405
658
72
2
101
38
24
9
84
None
None
3
None
49
None
114
None
3636
3
148
2631
7
575
None
2616
73
70
2968
25
653
76
3
5
None
701
58
28
None
4
24
884
4
4
319
297
None
68
845
None
1660
4
None
537
13
156
160
609
77
None
399
1898
None
372
528
449
1089
None
411
None
918
446
568
61
461
70
1162
1463
164
384
5
51
57
118
55
443
58
12
122
2
7
42
1992
155
129
118
260
383
62
92
68
7
138
65
222
620
71
46
73
2
None
1887
None
8
25
None
637
105
None
183
116
335
180
None
16
None
98
101
407
None
764
16
278
2393
157
25
249
2
2034
18
60
230
13
43
None
None
1679
67
None
14
None
170
55
136
8
194
None
158
66
160
369
290
2

In [26]:
X["train"].shape[0]

9025

In [27]:
sort_dev_k = np.array(app_k_nonon, dtype=int).argsort()
sort_dev_k

array([473, 513, 938, ..., 841, 823, 686])

In [28]:
app_k_nonon[253]

9025

In [29]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [30]:
new_predictions = np.load("new_predictions.npy", allow_pickle=True)

In [31]:
data = []
for test_movie_id in sort_dev_k:
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if k == None:
        test_point.append(None)
        data.append(test_point)
        continue
    k = int(k)
    test_point.append(np.round(new_predictions[test_movie_id]) != np.round(pred[test_movie_id].item()))
        
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)

k 1
k 1
k 1
k 1
k 1
k 1
k 2
k 2
k 2
k 2
k 2
k 2
k 2
k 2
k 2
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 3
k 4
k 4
k 4
k 4
k 4
k 4
k 4
k 4
k 4
k 5
k 5
k 5
k 5
k 5
k 5
k 6
k 6
k 6
k 6
k 6
k 6
k 7
k 7
k 7
k 7
k 7
k 7
k 8
k 8
k 8
k 8
k 8
k 8
k 9
k 9
k 9
k 9
k 9
k 9
k 10
k 10
k 10
k 11
k 12
k 12
k 12
k 12
k 12
k 13
k 13
k 13
k 13
k 13
k 13
k 13
k 13
k 14
k 15
k 15
k 15
k 16
k 16
k 16
k 16
k 16
k 16
k 16
k 17
k 17
k 17
k 17
k 17
k 17
k 17
k 18
k 18
k 18
k 19
k 19
k 19
k 20
k 20
k 21
k 21
k 21
k 21
k 22
k 22
k 22
k 22
k 23
k 23
k 24
k 24
k 24
k 24
k 24
k 24
k 24
k 24
k 25
k 25
k 25
k 25
k 25
k 25
k 26
k 26
k 26
k 26
k 26
k 27
k 27
k 27
k 27
k 27
k 28
k 28
k 28
k 28
k 28
k 29
k 29
k 30
k 30
k 31
k 32
k 32
k 32
k 32
k 33
k 33
k 33
k 34
k 35
k 35
k 35
k 36
k 36
k 37
k 37
k 37
k 37
k 38
k 38
k 38
k 39
k 39
k 39
k 39
k 40
k 41
k 41
k 42
k 42
k 42
k 43
k 43
k 43
k 43
k 43
k 43
k 44
k 44
k 44
k 44
k 45
k 45
k 45
k 45
k 46
k 46
k 46
k 46
k 47
k 47
k 48
k 48
k 48
k 49
k 49
k 49
k 49
k 50
k 50
k 50
k 51


In [32]:
import csv

with open('CSV_essay_bertlin_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [43]:
ls

CSV_SST_LR_alg1.csv    IP_change_log.npy      difference_k_appk.npy
CSV_SST_LR_alg1.ipynb  IP_flip_log.npy        exact_ks_IP.npy
Errors_k.ipynb         Plot.ipynb             get_order.ipynb
GC_change.npy          RIF_change.npy         need_reg?.ipynb
GC_flip.npy            RIF_flip.npy           new_pred_k.npy
GD_change.npy          Same.jpg               new_predictions.npy
GD_flip.npy            Smallest_k.ipynb       old_predictions.npy
IF_change_log.npy      Use_approK.ipynb       order_IF.npy
IF_flip_log.npy        appro_ks_IP.npy        pred.npy


In [44]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_SST/IF_lr_fix'